In [1]:
%pip install xgboost


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import xgboost as xgb
print("XGBoost imported successfully!")

XGBoost imported successfully!


In [90]:
import sys
print(sys.executable)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import xgboost as xgb
import warnings

# Suppress specific warnings from xgboost
warnings.filterwarnings(action='ignore', category=UserWarning, message='.*gpu_id.*')
warnings.filterwarnings(action='ignore', category=UserWarning, message='.*gpu_hist.*')

# Load data
file_path = r"/home/innobridge/sherry/Final_Balanced_Data.csv"
data = pd.read_csv(file_path)

# Define poverty threshold and create target variable
poverty_threshold = 13590
data['poverty_status'] = np.where(data['PINCP'] <= poverty_threshold, 1, 0)

# Drop the PINCP column
data = data.drop(columns=['PINCP'])

# Split data into features and target
X = data.drop(columns=['poverty_status'])
Y = data['poverty_status']

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=18)

# Define parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 500, 800],             
    'max_depth': [7, 8, 9],   
    'learning_rate': [0.1, 0.2],   
    'min_child_weight': [2, 3],
  'gamma': [0], 
    'subsample': [1.0],
    'colsample_bytree': [0.6],
    'colsample_bylevel': [1.0],
    'colsample_bynode': [1.0],
   'reg_alpha': [10],
    'reg_lambda': [1],
    'max_delta_step': [0],
    'scale_pos_weight': [1]
}

# Initialize XGBoost classifier without use_label_encoder, using GPU
xgb_model = xgb.XGBClassifier(random_state=18, eval_metric='logloss')

# Perform grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
grid_search.fit(X_train, Y_train)

# Get the best model
best_xgb_model = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# Make predictions with the best model
predictions = best_xgb_model.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(Y_test, predictions)
precision = precision_score(Y_test, predictions)
recall = recall_score(Y_test, predictions)
f1 = f1_score(Y_test, predictions)
conf_matrix = confusion_matrix(Y_test, predictions)

# Print evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix: \n{conf_matrix}")

/home/innobridge/anaconda3/bin/python
Fitting 3 folds for each of 48 candidates, totalling 144 fits
Best parameters found:  {'colsample_bylevel': 1.0, 'colsample_bynode': 1.0, 'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 8, 'min_child_weight': 2, 'n_estimators': 500, 'reg_alpha': 10, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1.0}
Accuracy: 0.8158421898615743
Precision: 0.830718789407314
Recall: 0.7921882703801243
F1 Score: 0.810996140517183
Confusion Matrix: 
[[70151 13424]
 [17281 65876]]
